In [ ]:
!pip install accelerate --quiet

from huggingface_hub import login

login(token="add your access token to hf here to run the model")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

2025-05-27 17:09:57.429563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748365797.620562      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748365797.675877      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [3]:
recipes = {
    "quick creamy lumps": {
        "file": "images/30-Minute_Chicken_and_Dumplings.jpg",
        "instruction": "Cook Chicken & Base: Sauté veggies, add broth and chicken, then boil. Add Dumplings: Drop in dumpling mix and steam. Finish: Stir in peas, thicken, and serve."
    },
    "buttery farm meat": {
        "file": "images/Amish_Chicken.jpg",
        "instruction": "Coat Chicken: Dredge chicken in seasoned flour and place in a dish. Add Cream: Pour mixed cream and water over it. Bake: Cook at 350°F until golden."
    },
    "soft creamy swirls": {
        "file": "images/Angel_Chicken_Pasta.jpg",
        "instruction": "Make Sauce: Melt butter, add dressing, wine, soup, cream cheese. Bake Chicken: Cover chicken with sauce, bake at 325°F for 60 mins. Serve: Cook pasta, serve chicken & sauce over it."
    },
    "soy noodle thing": {
        "file": "images/Asian_Noodle_Bowl.jpg",
        "instruction": "Prep & Toast: Boil water. Toast almonds until golden. Cook Pasta & Veggies: Cook pasta; add broccoli, then snow peas & peppers towards the end. Sauce & Serve: Whisk almond butter, soy, lime, sugar, chili-garlic sauce & pasta water. Toss with drained pasta/veggies; garnish with almonds & scallions."
    },
    "spicy bean mess": {
        "file": "images/Award_Winning_Chili.jpg",
        "instruction": "Heat oil, brown ground meat. Add onions, peppers, jalapeno; cook until soft. Drain grease. Stir in tomatoes, sauce, spices, water, and beans. Simmer covered for 1-2 hours. Serve with your favorite toppings"
    },
    "pink baked slab": {
        "file": "images/Baked_Salmon.jpg",
        "instruction": "Mix garlic, oil, basil, salt, pepper, lemon juice, parsley. Marinate salmon in fridge for 1 hour. Preheat oven to 375°F. Seal salmon and marinade in foil. Place foil-sealed salmon in dish, bake 35-45 minutes until flaky."
    },
    "dark sweet thighs": {
        "file": "images/Balsamic_Chicken_Thighs.jpg",
        "instruction": "Prep & Brown: Preheat sprayed pan. Season chicken thighs and brown well on all sides. Cook & Add Aromatics: Cover, reduce heat, cook ~25 mins until done. Add and soften shallots. Sauce & Serve: Stir in balsamic vinegar, coat chicken, and serve with sauce spooned over."
    },
    "beefy crunchy pie": {
        "file": "images/Beef_Nacho_Crescent_Bake.jpg",
        "instruction": "Preheat oven to 375°F, grease dish. Cook beef with onion, garlic, pepper, and spices; drain. Mix soup & milk. Roll crescent dough, cut into squares, fill with beef, seal, and place in dish; drizzle with soup mix. Bake 20-25 mins until golden. Sprinkle with cheese, bake 5 more mins until melted."
    },
    "fluffy malt loaf": {
        "file": "images/Beer_Bread.jpg",
        "instruction": "Mix & Bake: Preheat oven to 375°F. Mix sifted dry ingredients with beer, pour into greased pan, top with melted butter, and bake for 1 hour. Sift Flour Crucial: For best results (not a \"brick\"), sift flour or spoon it lightly into the measuring cup.... Tips & Variations: For softer crust, mix butter into batter. If using non-alcoholic liquid, add yeast."
    },
    "tomato bread bites": {
        "file": "images/Best_Ever_Bruschetta.jpg",
        "instruction": "Mix Ingredients: Combine tomatoes, garlic, basil, vinegar, olive oil, cheese, salt, and pepper in a bowl. Marinate: Let the mixture sit for at least 15 minutes to blend flavors. Serve: Slice bread and spoon the tomato mixture on top."
    },
    "garlicky red splash": {
        "file": "images/Bevs_Spaghetti_Sauce.jpg",
        "instruction": "Brown beef, onion, garlic with olive oil and all seasonings (bay leaves, oregano, basil, Italian seasoning, salt, pepper). Stir in tomato paste, sauce, and diced tomatoes; simmer covered for 1.5 hours. Top cooked spaghetti with sauce, sautéed mushrooms, and Parmesan."
    },
    "sweet sticky meat": {
        "file": "images/Bourbon_Chicken.jpg",
        "instruction": "Brown Chicken: Heat oil, lightly brown chicken pieces, then remove from skillet. Make Sauce & Simmer: Add remaining ingredients to skillet, heat until dissolved. Return chicken, bring to a boil, then simmer for 20 minutes. Serve: Serve over hot rice."
    },
    "green cheesy bowl": {
        "file": "images/Broccoli_Cheese_Soup.jpg",
        "instruction": "Roux & Base: Sauté onions. Make a golden roux with butter/flour, then whisk in stock and simmer. Veggies & Cream: Add broccoli, carrots, onions, milk, and half & half; cook on low (do not boil). Cheese & Serve: Season, blend if desired, then stir in cheese until melted. Serve immediately."
    },
    "juicy charred slab": {
        "file": "images/Broil_a_Perfect_Steak.jpg",
        "instruction": "Prep & Preheat: Bring steaks to room temp, pat dry. Preheat oven and cast iron skillet under broiler for 15-20 mins. Season steaks. Sear & Bake: Carefully place steaks in hot skillet, broil 3 mins per side. Switch oven to 500°F, cook to desired doneness (see chart), flipping halfway. Rest & Serve: Rest steaks for 5 minutes before serving, ideally on warm plates. (Crucial: Don't pierce meat during cooking)."
    },
    "cheesy tuber meat": {
        "file": "images/Cheese_Potato_and_Sausage_Casserole.jpg",
        "instruction": "Prep Meat & Potatoes: Brown chopped smoked sausage. Combine with cooked, diced potatoes in a casserole dish. Make Cheese Sauce: Mix and heat remaining ingredients (except cheddar/paprika) until smooth; pour over sausage/potatoes. Top & Bake: Sprinkle with cheddar and paprika. Bake at 350°F for 35-45 minutes until golden."
    }
}

image_dir = '/kaggle/input/intern-data-new/images (2)/'
messages = []


messages.append({
    "role": "system",
    "content": [{
        "type": "text",
        "text": """You are an Expert Culinary Assistant. Your sole task is to convert a food image and a vague or imprecise dish title into an ultra-concise, 2-3 step recipe summary.

**Your Response MUST Adhere Strictly to These Rules:**

1.  **Structure & Length:**
    * **Exactly 2 or 3 steps.** No more, no less.
    * Each step must be a **single, complete sentence.**

2.  **Content & Style:**
    * Describe essential cooking actions using **present-tense, imperative verbs** (e.g., "Mix ingredients...", "Sauté vegetables...", "Bake until golden...").
    * Focus entirely on the **core assembly and cooking method.** What does one *do* to make this?

3.  **What to OMIT (Critically Important):**
    * **NO detailed ingredient lists.** (e.g., AVOID: "Take 2 cups flour, 1 cup sugar...")
    * **NO precise quantities, measurements, or extensive lists of minor seasonings.**
    * **NO specific cooking times or temperatures,** UNLESS it's a defining characteristic of the technique itself (e.g., "Pressure cook for 10 minutes," "Chill for 4 hours," "Deep fry at 375°F"). If not absolutely critical to conveying the *method*, omit it.
    * **NO instructions on plating, garnishing, or serving suggestions.**
    * **NO introductory phrases, chit-chat, or concluding remarks.** Get straight to the steps.

4.  **Goal:** The recipe summary must be **clear, unambiguous, actionable,** and capture the fundamental essence of how the dish is prepared. Assume the user has basic cooking knowledge and can infer common sense details.
"""
    }]
})

for vague_title, data in recipes.items():
    img = Image.open(f"{image_dir}/{data['file']}")
    messages.append({
        "role": "user",
        "content": [
            {"type": "image", "image": img},
            {"type": "text", "text": f'Vague Title: "{vague_title}"\nInstruction:'}
        ]
    })
    messages.append({
        "role": "assistant",
        "content": [
            {"type": "text", "text": data['instruction']}
        ]
    })

In [4]:

test_image_path = f"{image_dir}images/Crusty_French_Bread.jpg"
test_vague_title = "crunchy lemon flakes"

test_img = Image.open(test_image_path)
messages.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [5]:
inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=50)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Prepare Coating: Combine breadcrumbs, parmesan, parsley, garlic, and seasonings in a shallow dish. Dip fish in egg, then dredge in breadcrumb mixture. Bake: Place coated fish on a baking sheet and bake at 400°F for 15-20 minutes until golden and cooked through. Serve: Serve with lemon wedges and rice.


In [6]:

test_image_path = f"{image_dir}/images/Creamy_Burrito_Casserole.jpg"
test_vague_title = "golden curry hug"

test_img = Image.open(test_image_path)
messages.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [7]:
inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=50)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

TemplateError: Conversation roles must alternate user/assistant/user/assistant/...

In [ ]:

test_image_path = f"{image_dir}/images/ENTER_IMAGE_NAME"
test_vague_title = "ENTER VAGUE TITLE"

test_img = Image.open(test_image_path)
messages.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [ ]:
inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=50)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)